In [9]:
import pandas as pd # Manejo de bases de datos
import geopandas as gpd # Manejo de bases de datos geográficas
import numpy as np # Funciones numéricas
import matplotlib.pyplot as plt # Gráficas
import seaborn as sns # Gráficas
import datetime as dt
import folium
import unicodedata
import geojson
import urllib, json
import base64

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.neural_network import MLPClassifier

from shapely.geometry import LineString, Point
from geopandas.tools import sjoin
from branca.colormap import linear

import dash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import visdcc
from waitress import serve

### Prepare data needed to create DashBoard

In [10]:
####################################################################################################
############## Read Bogota spatial characteristics previously prepared and organized.###############
####################################################################################################
path = '/home/ubuntu/javeriana/MOTUS-PUJ/Step_2/2_ML_Preps/Outputs/partial_spatial_utam.pkl'
spatial_utam = pd.read_pickle(path, compression='gzip')
spatial_utam = gpd.GeoDataFrame(spatial_utam)
spatial_utam.crs = 'EPSG:4326'
spatial_utam.to_crs(epsg=4326, inplace=True)

###################################################################################################
#########################Estimate UTAM population weight vector####################################
###################################################################################################
def Arreglar_tilde(Texto):
    Texto = unicodedata.normalize('NFD', Texto)
    Texto = Texto.encode('ascii', 'ignore')
    Texto = Texto.decode("utf-8")
    Texto = Texto.lower()
    return(Texto)

# Bogotá localities population
path = '/home/ubuntu/javeriana/MOTUS-PUJ/Step_1/Files/Poblacion_Bogota.xlsx'
PobBog = pd.read_excel(path, sheet_name='1.1')
PobBog = PobBog.drop([PobBog.index[0], PobBog.index[1], PobBog.index[2], PobBog.index[3], PobBog.index[4], 
                      PobBog.index[25], PobBog.index[26]])
#Rename columns
PobBog = PobBog.filter(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 17', 'Unnamed: 18'])
PobBog = PobBog.rename(columns={'Unnamed: 0': 'id localidad', 'Unnamed: 1': 'localidad',
                                     'Unnamed: 17': 'Población 2020', 'Unnamed: 18': 'Población 2021'})
#Sorting Dataframe by locality id 
PobBog['id localidad'] = pd.to_numeric(PobBog['id localidad'])
PobBog.sort_values(by='id localidad', ascending=True)
PobBog.drop(columns={'Población 2020'}, inplace=True)
PobBog.reset_index(drop=True, inplace=True)
#Lower case and taking off accent marks
PobBog['localidad'] = PobBog.apply(lambda row: Arreglar_tilde(row['localidad']), axis=1)
pop_list = PobBog['Población 2021'].tolist()

# Localities codes are ascending numbers from 1 to 19
Loc_code = list(range(1,20))

LocHomes = []
for i in range(len(Loc_code)):
    #For each locality count the number of homes within it
    temp = spatial_utam[ spatial_utam['LOCid'] == Loc_code[i] ]
    temp_list = temp['HOGARES'].values.tolist()
    tot_homes = sum(temp_list)
    LocHomes.append(tot_homes)
    
ppl_weight = []
#Estimate the mean number of people each home needs to match locality population
for i in range(len(LocHomes)):
    temp = pop_list[i]/LocHomes[i]
    ppl_weight.append(temp)
    
del LocHomes, PobBog

###################################################################################################
############################Load RT estimated for each locality####################################
###################################################################################################
base_path = '/home/ubuntu/javeriana/MOTUS-PUJ/Step_1/RT_outputs/'

loc_R_list = ['usaquen.pkl', 'chapinero.pkl', 'santafe.pkl', 'sancristobal.pkl', 'usme.pkl',
             'tunjuelito.pkl', 'bosa.pkl', 'kennedy.pkl', 'fontibon.pkl', 'engativa.pkl',
             'suba.pkl', 'barriosunidos.pkl', 'teusaquillo.pkl', 'losmartires.pkl', 'antonionariño.pkl',
             'puentearanda.pkl', 'lacandelaria.pkl', 'rafaeluribeuribe.pkl', 'ciudadbolivar.pkl']

R_list = []

for i in range(len(loc_R_list)):
    path_file = base_path+loc_R_list[i]
    R_list.append(pd.read_pickle(path_file))
    
for i in range(len(loc_R_list)):
    R_list[i].reset_index(drop=False, inplace=True)
    
R_df = pd.DataFrame(index = R_list[0]['Time Stamp'])
R_df.reset_index(drop=False, inplace=True)

for i in range(len(loc_R_list)):
    R_df[loc_R_list[i]] = 0
    R_df[loc_R_list[i]] = R_list[i]['R'].tolist()
    
###################################################################################################
###########################Active cases count for each locality####################################
###################################################################################################
#fechas_df = pd.read_csv('/home/ubuntu/javeriana/MOTUS-PUJ/Bog_Estimation/ECM/fechas_sintomas.csv')
fechas_df = pd.read_csv('/home/ubuntu/javeriana/MOTUS-PUJ/Step_1/Outputs/fechas_sintomas.csv')
fechas_df = fechas_df.rename(columns={'LOCALIDAD_ASIS': 'CASOS_REPORTADOS'})#Rename Column

#Change str to datetime objects
fechas_df['FECHA_DE_INICIO_DE_SINTOMAS'] = pd.to_datetime(fechas_df['FECHA_DE_INICIO_DE_SINTOMAS'], format='%Y-%m-%d')
fechas_df = fechas_df.sort_values(by='FECHA_DE_INICIO_DE_SINTOMAS', ascending=True)
fechas_df = fechas_df.reset_index(drop=True)

#Drop 0 and 21 id codes since they don´t belong to Bogotá geography 
index_0 = fechas_df[ fechas_df['CODIGO_LOCALIDAD'] == 0 ].index
index_21 = fechas_df[ fechas_df['CODIGO_LOCALIDAD'] == 21 ].index

fechas_df.drop(index_0, inplace = True)#drop fuera de bogotá and sin dato rows since we cannot calculate infection 
fechas_df.drop(index_21, inplace = True)# density for them (we don´t know their reference population)

fechas_df = fechas_df.reset_index(drop = True)

com_index = len(fechas_df['FECHA_DE_INICIO_DE_SINTOMAS']) #column length 
start_date = fechas_df.loc[0, 'FECHA_DE_INICIO_DE_SINTOMAS'] #start of the pandemic in Bogotá 
end_date = fechas_df.loc[com_index-1, 'FECHA_DE_INICIO_DE_SINTOMAS'] #Last reported date 

pan_days = end_date - start_date
pan_days = int(pan_days.days) #Days passed since pandemic start to last reported date grid x axis 
local_bog = 20 # number of rows y axis we are not considering 0 and 21 codes 

#create grid and fill it with reported cases y axis correspond to Bogota localities id code, x axis correspond to 
#number of days passed since pandemic started that way 0 index -> 2020-02-06, 1->2020-02-07 and so on

grid = np.ndarray([local_bog, pan_days+1]) #create grid
grid.fill(0) #fill grid with 0 

#Method used to fill grid with reported cases 
def fillGrid(date, code, cases):
    col = date - start_date
    col = int(col.days)
    grid[code-1][col] = cases
    

#Fill grid with cases 
fechas_df.apply(lambda row: fillGrid(row['FECHA_DE_INICIO_DE_SINTOMAS'], row['CODIGO_LOCALIDAD'], int(row['CASOS_REPORTADOS']) ), axis=1)
#function to count active cases in determined date
def ActiveCases (date, code):
    col = date - start_date
    col = int(col.days)
    if col >= 15:
        Active = sum(grid[code-1][col-15:col+1])
    else: 
        Active = sum(grid[code-1][0:col+1])
    return int(Active)

c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:22: FutureWarning:

Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.



### Functions

In [11]:
def single_cases(weight, caseLoc, PobLoc, ID, Hog):
    x = (100*caseLoc)/(PobLoc)
    y = (x*weight[ID-1]*Hog)/100
    return y

def singularizeRT(loc, estrato, popden, hosp, ips, itur, sitp, comer, trips, cases, RT):
    if loc == 'candelaria':
        return RT
    else:
        a_rt = RT-estrato+popden-hosp-ips+itur+sitp+comer+trips+cases
        return a_rt
    
def ClusteringPrepDF(selec_date, selec_str, spatial_utam):
    #Add Active cases in interest date at locality level
    Active_cases = []
    for i in range(len(Loc_code)):
        Active_cases.append(ActiveCases(selec_date, Loc_code[i]))
        
    #Add Active Cases
    spatial_utam_date = spatial_utam.copy()    
    spatial_utam_date['cases'+selec_str] = spatial_utam_date.apply(lambda row: Active_cases[row['LOCid']-1], axis=1)
    #Estimate Active cases in a lower geographical level
    spatial_utam_date['UTAM_cases'+selec_str] = spatial_utam_date.apply(lambda row: single_cases(ppl_weight, 
                                                                                                  row['cases'+selec_str], 
                                                                                                  row['PopLoc'], row['LOCid'], 
                                                                                                  row['HOGARES']), axis=1)
    #Locality RT in interest date
    rt_list = R_df[ R_df['Time Stamp'] == selec_date ]
    rt_list = rt_list.iloc[:, 1:].values.tolist()[0]
    #Add RT to DataFrame
    spatial_utam_date['Rt'+selec_str] = spatial_utam_date.apply(lambda row: rt_list[row['LOCid']-1], axis=1)
    #Estimate RT in a lower geographical level
    spatial_utam_date['Total trips'] = spatial_utam_date['originated Trips']+spatial_utam_date['received Trips']
    spatial_utam_date.drop(columns={'HOGARES', 'UTAMArea', 'originated Trips', 'received Trips',
                               'N_PlazMer', 'N_Col'}, inplace=True)
    
    spatial_utam_date_rt = spatial_utam_date.copy()
    spatial_utam_date_rt['ESTRATOPre'] = spatial_utam_date_rt['ESTRATOPre']/(spatial_utam_date_rt['ESTRATOPre'].max()*4)
    spatial_utam_date_rt['PopDen[p/km2]'] = spatial_utam_date_rt['PopDen[p/km2]']/(spatial_utam_date_rt['PopDen[p/km2]'].max()*4)
    spatial_utam_date_rt['N_Hosp'] = spatial_utam_date_rt['N_Hosp']/(spatial_utam_date_rt['N_Hosp'].max()*4)
    spatial_utam_date_rt['N_IPS'] = spatial_utam_date_rt['N_IPS']/(spatial_utam_date_rt['N_IPS'].max()*4)
    spatial_utam_date_rt['N_ITur']  = spatial_utam_date_rt['N_ITur']/(spatial_utam_date_rt['N_ITur'].max()*4)
    spatial_utam_date_rt['N_SITP'] = spatial_utam_date_rt['N_SITP']/(spatial_utam_date_rt['N_SITP'].max()*4)
    spatial_utam_date_rt['N_Ecomer'] = spatial_utam_date_rt['N_Ecomer']/(spatial_utam_date_rt['N_Ecomer'].max()*4)
    spatial_utam_date_rt['Total trips'] = spatial_utam_date_rt['Total trips']/(spatial_utam_date_rt['Total trips'].max()*4)
    spatial_utam_date_rt['UTAM_cases'+selec_str] = spatial_utam_date_rt['UTAM_cases'+selec_str]/(spatial_utam_date_rt['UTAM_cases'+selec_str].max()*3)
    
    spatial_utam_date_rt['singleRT_'+selec_str] = spatial_utam_date_rt.apply(lambda row: singularizeRT(row['LOCNombre'], row['ESTRATOPre'], 
                                                                                       row['PopDen[p/km2]'], row['N_Hosp'],
                                                                                       row['N_IPS'], row['N_ITur'],
                                                                                       row['N_SITP'], row['N_Ecomer'],
                                                                                       row['Total trips'], 
                                                                                       row['UTAM_cases'+selec_str],
                                                                                       row['Rt'+selec_str]), axis=1)
    
    # Finally generate final normalized DF with an estimated rt for each utam 
    spatial_utam_date['local_rt_'+selec_str] = spatial_utam_date_rt['singleRT_'+selec_str]

    spatial_utam_date['ESTRATOPre'] = spatial_utam_date['ESTRATOPre']/(spatial_utam_date['ESTRATOPre'].max())
    spatial_utam_date['PopDen[p/km2]'] = spatial_utam_date['PopDen[p/km2]']/(spatial_utam_date['PopDen[p/km2]'].max())
    spatial_utam_date['N_Hosp'] = spatial_utam_date['N_Hosp']/(spatial_utam_date['N_Hosp'].max())
    spatial_utam_date['N_IPS'] = spatial_utam_date['N_IPS']/(spatial_utam_date['N_IPS'].max())
    spatial_utam_date['N_ITur']  = spatial_utam_date['N_ITur']/(spatial_utam_date['N_ITur'].max())
    spatial_utam_date['N_SITP'] = spatial_utam_date['N_SITP']/(spatial_utam_date['N_SITP'].max())
    spatial_utam_date['N_Ecomer'] = spatial_utam_date['N_Ecomer']/(spatial_utam_date['N_Ecomer'].max())
    spatial_utam_date['Total trips'] = spatial_utam_date['Total trips']/(spatial_utam_date['Total trips'].max())
    spatial_utam_date['UTAM_cases'+selec_str] = spatial_utam_date['UTAM_cases'+selec_str]/(spatial_utam_date['UTAM_cases'+selec_str].max())
    spatial_utam_date['local_rt_'+selec_str] = spatial_utam_date['local_rt_'+selec_str]/(spatial_utam_date['local_rt_'+selec_str].max())
    
    spatial_utam_date.drop(columns={'Rt'+selec_str, 'cases'+selec_str}, inplace=True)
    spatial_utam_date = spatial_utam_date[ ['LOCNombre', 'PopLoc', 'UTAM', 'LOCid', 'ESTRATOPre', 'UTAMNombre',
                                       'PopDen[p/km2]', 'geometry', 'N_Hosp', 'N_IPS', 'N_ITur', 'N_SITP',
                                       'N_Ecomer', 'UTAM_cases'+selec_str, 'local_rt_'+selec_str ,'Total trips'] ]
    return spatial_utam_date

def StratRisk(Cluster):
    if Cluster == 0:
        return 0
    elif Cluster == 1:
        return 2
    elif Cluster == 2:
        return 1
    
def Train_Clustering(spatial_utam):
    selec_date = '10/01/2021'
    selec_str = '10-01-2021'
    selec_date = dt.datetime.strptime(selec_date, "%d/%m/%Y")

    TrainDF = ClusteringPrepDF(selec_date, selec_str, spatial_utam)
    TrainDF_mat = TrainDF.drop(columns={'LOCNombre', 'PopLoc','UTAM', 'LOCid', 'UTAMNombre', 'geometry', 
                                         'ESTRATOPre', 'PopDen[p/km2]', 'N_Hosp', 'N_IPS', 'N_ITur', 'N_SITP', 
                                         'N_Ecomer', 'Total trips'})
    TrainMat = TrainDF_mat.to_numpy()

    # Cluster characteristics
    n_clust = 3
    # Train Cluster Kmeans
    global_kmeans = KMeans(n_clusters=n_clust, random_state=10).fit(TrainMat)
    return global_kmeans

def Calc_date(days_p, mode):
    start = start = dt.datetime.strptime('26/3/2020', "%d/%m/%Y")
    actual = start + dt.timedelta(days=days_p)
    if mode == 0:
        return actual
    elif mode == 1:
        return ( str(actual.day)+'-'+str(actual.month)+'-'+str(actual.year) )
    
    elif mode == 2:
        return ( 'Day: '+str(actual.day)+' Month: '+str(actual.month)+' Year: '+str(actual.year) )
        

def Strat_Risk_Map(date, spatial_utam, global_kmeans):
    selec_date = Calc_date(date, 0)
    selec_str  = Calc_date(date, 1)

    RiskDF = ClusteringPrepDF(selec_date, selec_str, spatial_utam)
    RiskDFmat = RiskDF.drop(columns={'LOCNombre', 'PopLoc','UTAM', 'LOCid', 'UTAMNombre', 'geometry', 
                                         'ESTRATOPre', 'PopDen[p/km2]', 'N_Hosp', 'N_IPS', 'N_ITur', 'N_SITP', 
                                         'N_Ecomer', 'Total trips'})
    RiskMat = RiskDFmat.to_numpy()
    #Predict Cluster label
    labels = global_kmeans.predict(RiskMat)
    RiskDF['kmeans_label_full'] = labels
    RiskDF['Risk'] = RiskDF.apply(lambda row: StratRisk(row['kmeans_label_full']), axis=1)
    
    #path = '/home/ubuntu/javeriana/MOTUS-PUJ/Localidades/Loca.shp'
    path = '/home/ubuntu/javeriana/MOTUS-PUJ/Step_2/1_spatial/spatial_features/Locality/Loca.shp'
    Localities = gpd.read_file(path)
    Localities = Localities.sort_values(by='LocCodigo', ascending=True)
    Localities = Localities.reset_index(drop=True)
    Localities['LocCodigo'] = pd.to_numeric(Localities['LocCodigo'], downcast='integer')
    Localities.to_crs(epsg=4326, inplace=True)
    Localities.drop(columns=['LocAAdmini', 'LocArea', 'SHAPE_Leng', 'SHAPE_Area'], inplace=True)
    Localities['LocNombre'] = Localities.apply(lambda row: Arreglar_tilde(row['LocNombre']), axis=1)
    Localities.drop(19, axis=0, inplace=True)
    
    #Init folium map object for Bogotá
    Lat = 4.61
    Long = -74.082
    m = folium.Map(location=[Lat, Long], zoom_start=12, tiles='CartoDB positron')
    
    feature_Localidad = folium.FeatureGroup(name='Localidades', overlay=True, show=False).add_to(m)
    feature_UTAM = folium.FeatureGroup(name='UTAM', overlay=False, show=True).add_to(m)
    
    #Add Localities layer
    folium.GeoJson(Localities,
    style_function=lambda feature: {
        'fillColor': 'white',
        'color': 'white',
        'weight': 2,
        'fillOpacity':0.2,
    },
    highlight_function=lambda x: {'weight':4,'fillColor':'white','color': 'white'},
                   tooltip=folium.GeoJsonTooltip(fields=['LocNombre'],
                                                labels=False,
                                                sticky=True),
    name='Localidades').add_to(feature_Localidad)
    
    #Add UTAM Risk Estimation Choropleth Layer
    choropleth1 = folium.Choropleth(
    geo_data=RiskDF,
    name='choropleth1',
    data=RiskDF,
    columns=['UTAMNombre', 'Risk'],
    key_on='feature.properties.UTAMNombre',
    fill_color='YlOrRd',
    nan_fill_color="white",
    fill_opacity=1,
    line_opacity=0.2,
    #legend_name='Risk Stratification',
    highlight=True,
    line_color='black').geojson.add_to(feature_UTAM)
    
    geojson1 = folium.GeoJson(data=RiskDF, 
                              name='UTAM', 
                              smooth_factor=2, 
                              style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5}, 
                              tooltip=folium.GeoJsonTooltip(fields=['UTAMNombre', 'Risk'],
                                                            aliases=['Name', 'Risk group'],
                                                            labels=True, 
                                                            sticky=True), 
                              highlight_function=lambda x: {'weight':3,'fillColor':'grey'}, 
                             ).add_to(choropleth1)
    #colormap = linear.YlOrRd_09.scale(0, 2).to_step(2)
    #colormap.caption = 'Strat_risk'
    #colormap.add_to(m)

    folium.TileLayer('cartodbdark_matter', overlay=True, name="night mode").add_to(m)
    folium.LayerControl(collapsed=False).add_to(m)
    
    return (m.get_root().render())




In [12]:
#Train Clustering
global_kmeans = Train_Clustering(spatial_utam)
app = dash.Dash(
    external_stylesheets=[dbc.themes.CYBORG]
)
image_filename = 'Com_Risk_Scale.png' #image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

### --- Principal Title --- ### 
app.title = 'PUJ MOTUS TG'
### ---------------- Logo - Image ------------------ ###

Arriba = dbc.NavbarSimple(children=[
html.Img(src='https://www.javeriana.edu.co/recursosdb/20129/601896/escudoPUJ-Bogota_rgb-azul_lateral.png/3eec1aa5-947a-1655-7fe7-6c4dfb397793?t=1611842495478',
         width=230,height=100)],
    brand="MOTUS Science - PUJ",
    brand_href="#",
    color="navy",
    dark=True,
)



Estilo_arriba_caja = {'textAlign': 'center','height': 75, 'display': 'block', 
                      'margin-left': 'auto','margin-right': 'auto','width': '100%', 
                      'color': '#ffffff',"background-color": 'navy'}

Estilo_caja = {"max-width": "2000px", 'height': 'calc(130vh - 60px)', 'overflowY': 'scroll',
               "opacity": 0.9, "background-color": "black",'textAlign': 'center', 'width': '100%'}

Cuerpo = html.Div([dbc.Toast([html.Div([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),width=480, height=140),
                                        html.Br(),
    dcc.Slider(
    id='dias_pandemia',min=0,
    max=550,
    value=0,
    marks={
        0:  {'label': '26/03/2020'},
        60: {'label': '25/05/2020'},
        128:{'label': '01/08/2020'},
        220:{'label': '01/11/2020'},
        290:{'label': '10/01/2021'},
        349:{'label': '10/03/2021'},
        401:{'label': '01/05/2021'},
        451:{'label': '20/06/2021'},
        524:{'label': '01/09/2021'}
    },
    included=False),html.Div(id='Resultados_mapa'),visdcc.Run_js(id='Correr_java')])],
                    header= 'Community risk stratification map for Bogotá divided by UTAM administrative units. Select the UTAM button to observe the color scale risk stratification, check or un-check the "Localidades" box to show localities divisons in Bogotá. The risk score is calculated considering RT score and active cases within selected date in a UTAM'
                             ,header_style = Estilo_arriba_caja,
                   style=Estilo_caja)],style={
            'marginTop': 30,
            'marginBottom': 20,
            'marginLeft': 250,
            'marginRight': 250,
            'padding': 0,'background': "#f5f5f5","opacity": 1,})



######### ---------------- Esto es un callback -------- ####### 

@app.callback(
    [Output('Resultados_mapa', 'children'),
     Output('Correr_java', 'run')],
    [Input('dias_pandemia', 'value')])

def Decision_analisis(dias_pandemia): 
    java = ''' alert("''' + 'Risk estimation calculated for date '+Calc_date(dias_pandemia, 2)+'''"); '''
    mapa = html.Section(
        children=[html.Iframe(id='Mapa_general', srcDoc=Strat_Risk_Map(dias_pandemia, spatial_utam, global_kmeans), 
                              height = 950, 
            style={'display': 'flex','width': '100%','border': 0,
               'top': 0,
               'left' : 0,
               'bottom': 0,
               'right': 0} )],
        style={
            'padding': 0,
            'margin': 0,'height': 'calc(100vh - 60px)', 'overflowY': 'scroll',
            'borderRadius': 0,
            'border': 'thin lightgrey ridge', 'border-color': '#c7bfbf', 'border-width': '0px',
            'background': '#f5f5f5',"opacity": 0.9
        })
    
    return(mapa,java)
    
########## ------------- Abajo ------------- ############# 
Abajo = html.Div([html.H6(
        children='All rights reserved - Copyright ©  2021.',
        style={'textAlign': 'center','color': 'white'})])

#app.layout = html.Div(children=[Arriba, Cuerpo, Abajo])
app.layout = html.Div(children=[Arriba, dcc.Loading(id="loading-1", type="circle", fullscreen=False, 
                                                                  children=[Cuerpo]), Abajo])


############# --------------- Server ----------- ############# 
#serve(app.server,host='127.0.0.1',port=8050) #this connects with the server

######## ------- Desplegar servidor -------- ########
#server = create_server(app,threaded=True)
#server.run()
app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Nov/2021 16:03:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 16:03:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 16:03:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 16:03:24] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Nov/2021 16:03:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 16:03:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 16:04:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 16:04:44] "POST /_dash-update-component HTTP/1.1" 200 -
